In [122]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 24.2 MB/s 


In [123]:
import pandas as pd
import numpy as np
import operator
from operator import itemgetter
from itertools import groupby
import re
import unidecode

In [2]:
dir = '/content/drive/MyDrive/20212/THDL/data/temp_csv/'
mytour_path = dir + 'All_data_mytour.csv'
traveloka_path = dir + 'All_data_traveloka.csv'
agoda_path = dir + 'agoda_final - agoda_final.csv'
booking_path = dir + 'craw_booking_com_full.csv'

# **Đọc dữ liệu**

In [3]:
mytour_df = pd.read_csv(mytour_path)
traveloka_df = pd.read_csv(traveloka_path)
agoda_df = pd.read_csv(agoda_path)
booking_df = pd.read_csv(booking_path)

In [4]:
booking_df.head(3)

,City,Hotel name,Url,Address,Stars,Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places,Image
0,Hà Nội,Khách sạn\r\nHANZ Regal Hotel Hanoi,https://www.booking.com/hotel/vn/90-pham-huy-t...,"90 Pham Huy Thong, Quận Ba Đình, Hà Nội, Viê...",2,VND 421.890,"7,8",86 đánh giá,"['“máy lạnh mát, wifi trong từng phòng nên mạn...","['Xe đưa đón sân bay', 'Wi-Fi miễn phí', 'P...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Hồ Ngọc Khánh', '0,3 km'), ('Trung tâm thươ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1,Hà Nội,Khách sạn\r\nLa Nueva Boutique Hotel Hanoi & Spa,https://www.booking.com/hotel/vn/hanoi-eleganc...,"32 Lo Su Street, Quận Hoàn Kiếm, Hà Nội, Viê...",4,VND 1.316.000,"9,3",209 đánh giá,"['“Địa điểm tuyệt vời, đi một xíu là ra đến hồ...","['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Nhà hát múa rối Thăng Long', '0,1 km'), ('Đ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2,Hà Nội,Khách sạn\r\nLa Storia Ruby Hotel,https://www.booking.com/hotel/vn/hanoi-eleganc...,"3 Yen Thai Street, Quận Hoàn Kiếm, Hà Nội, V...",3,VND 625.000,"9,6",217 đánh giá,['“- vị trí gần hồ gươm rất tiện cho việc đi b...,"['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Quảng Trường Đông Kinh Nghĩa Thục', '0,4 km...",['https://cf.bstatic.com/xdata/images/hotel/ma...


In [5]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4876 entries, 0 to 4875
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   City               4876 non-null   object
 1   Hotel name         4876 non-null   object
 2   Url                4876 non-null   object
 3   Address            4876 non-null   object
 4   Stars              4876 non-null   int64 
 5   Price              4876 non-null   object
 6   Rating             4876 non-null   object
 7   Number of reviews  4875 non-null   object
 8   Reviews            4876 non-null   object
 9   Facilities         4876 non-null   object
 10  Description        4876 non-null   object
 11  Nearby places      4876 non-null   object
 12  Image              4876 non-null   object
dtypes: int64(1), object(12)
memory usage: 495.3+ KB


In [6]:
agoda_df.head(3)

,city,hotel name,url,address,stars,price,rating,location rating,n_reviews,image,reviews,facilities,nearby places
0,Nha Trang,Nhà Dân Megi (Megi Homestay),https://www.agoda.com/vi-vn/megi-home-hostel-h...,"7 Sao Biển Vĩnh Hải, Nha Trang, Khánh Hòa., Vĩ...",NaN,81.818,"9,8",NaN,93 Nhận xét,https://pix6.agoda.net/hotelImages/11029619/-1...,[],"['Dọn phòng hằng ngày', 'câu cá', 'sân vườn', ...","['Viện Hải dương học - 7,91 km', 'Đại học Nha ..."
1,Nha Trang,Căn Hộ Star Beach (Star Beach Apartment),https://www.agoda.com/vi-vn/star-beach-nha-tra...,"02 Nguyen Thi Minh Khai Street, Lộc Thọ, Nha T...",5.0,445.886,"9,6","9,9",127 Nhận xét,https://pix6.agoda.net/hotelImages/134/1348629...,"['Địa điểm đẹp và khách sạn đẹp, rộng rãi, vie...","['Dọn phòng hằng ngày', 'Điều hòa', 'Đưa đón s...","['Hon Chong - 3,9 km', 'Bảo tàng Hải dương học..."
2,Nha Trang,Khách Sạn Mojzo (Mojzo Inn),https://www.agoda.com/vi-vn/mojzo-inn_2/hotel/...,65/07 Nguyen Thiet Thuat- Nha Trang- Khanh hoa...,NaN,194.567,"9,6",NaN,1.806 Nhận xét,https://pix6.agoda.net/hotelImages/5945444/-1/...,[],"['Điều hòa', 'Đưa đón sân bay', 'Cách bãi biển...","['Hon Chong - 3,99 km', 'i resort - 4,16 km', ..."


In [7]:
agoda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9031 entries, 0 to 9030
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             9031 non-null   object 
 1   hotel name       8997 non-null   object 
 2   url              9031 non-null   object 
 3   address          6214 non-null   object 
 4   stars            8839 non-null   float64
 5   price            8757 non-null   object 
 6   rating           5128 non-null   object 
 7   location rating  1054 non-null   object 
 8   n_reviews        9031 non-null   object 
 9   image            7952 non-null   object 
 10  reviews          9031 non-null   object 
 11  facilities       9031 non-null   object 
 12  nearby places    9031 non-null   object 
dtypes: float64(1), object(12)
memory usage: 917.3+ KB


In [8]:
mytour_df.head(3)

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
0,Hà Nội,Khách Sạn Apricot Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/954-khach-san-apri...,"136, Hàng Trống, Quận Hoàn Kiếm, Hà Nội, Việt Nam",5,1.908.235đ,9.2,320,['Nhận xét chất lượng phòng\nNV lịch sự. Chất ...,"['American Express', 'Lễ tân 24h', 'Bán tour',...",['Khách sạn Apricot Hotel mang vẻ đẹp lỗng lẫy...,"[('Kanko Izakaya', '60 m'), ('Nhà hàng Lục Thủ..."
1,Hà Nội,Khách Sạn Imperial Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/972-khach-san-impe...,"44, Hàng Hành, Quận Hoàn Kiếm, Hà Nội, Việt Nam",0,852.946đ,9.4,159,['Đánh giá trải nghiệm khách sạn\nKhách sạn đẹ...,"['Lễ tân 24h', 'Trợ giúp đặc biệt', 'Giữ hành ...",['Imperial Hotel Hanoi đi và hoạt động từ năm ...,"[('Nhà hàng và cafe Old Street', '25 m'), ('Po..."
2,Hà Nội,Khách Sạn Beryl Palace Hanoi,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/49198-khach-san-be...,"173a, Hàng Bông, Quận Hoàn Kiếm, Hà Nội, Việt Nam",4,638.138đ,9.2,42,['Trải nghiệm tuyệt vời!\nMình đi nghỉ với gia...,"['Đưa/đón khách sân bay', 'Cho thuê xe đạp', '...",['Lối trang trí kiểu cổ điển của Pháp và kiến ...,"[('O Lounge, Bistro and Sport Bar', '9 m'), ('..."


In [9]:
mytour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1295 non-null   object 
 1   Hotel name         1290 non-null   object 
 2   Image              1295 non-null   object 
 3   Url                1295 non-null   object 
 4   Address            1290 non-null   object 
 5   Stars              1295 non-null   int64  
 6   Min Price          1051 non-null   object 
 7   Rating             1290 non-null   float64
 8   Number of reviews  1295 non-null   int64  
 9   Reviews            1295 non-null   object 
 10  Facilities         1295 non-null   object 
 11  Description        1295 non-null   object 
 12  Nearby places      1295 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 131.6+ KB


In [10]:
traveloka_df.head(3)

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
0,Hà Nội,Hanoi Boutique Hotel,[],https://www.traveloka.com/vi-vn/hotel/vietnam/...,"30/6 Lò Sũ, Lý Thái Tổ, Quận Hoàn Kiếm, Hà Nộ...",3,475.000 VND,8.7,77,"['Phòng rộng và sạch sẽ, khách sạn ngay tại tr...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Nhà hát Múa rối Nước Thăng Long', '179 m'),..."
1,Hà Nội,Khách sạn MK Premier Boutique,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"72-74 Hàng Buồm, Hàng Buồm, Quận Hoàn Kiếm,...",4,616.173 VND,8.7,113,"['khách sạn ở hành lang không có điều hòa, đườ...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Đền Bạch Mã', '12 m'), ('Hanoi Train Street..."
2,Hà Nội,Classy Holiday Hotel & Spa,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"49 Lãn Ông, Hàng Bồ, Quận Hoàn Kiếm, Hà Nội, ...",4,487.800 VND,9.4,177,['Phục vụ dọn phòng mở cửa phòng không gọi cửa...,"['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('King Pirates Pub', '229 m'), ('Hanoi Train ..."


In [11]:
traveloka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1116 non-null   object 
 1   Hotel name         1116 non-null   object 
 2   Image              1116 non-null   object 
 3   Url                1116 non-null   object 
 4   Address            1107 non-null   object 
 5   Stars              1116 non-null   int64  
 6   Min Price          1116 non-null   object 
 7   Rating             1116 non-null   float64
 8   Number of reviews  1116 non-null   int64  
 9   Reviews            1116 non-null   object 
 10  Facilities         1116 non-null   object 
 11  Description        1116 non-null   object 
 12  Nearby places      1116 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 113.5+ KB


# **Missing value**

In [12]:
#Booking
percent_missing = booking_df.isnull().sum() * 100 / len(booking_df)
missing_value_df = pd.DataFrame({'column_name': booking_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
City,City,0.000000
Hotel name,Hotel name,0.000000
Url,Url,0.000000
Address,Address,0.000000
Stars,Stars,0.000000
Price,Price,0.000000
Rating,Rating,0.000000
Number of reviews,Number of reviews,0.020509
Reviews,Reviews,0.000000
Facilities,Facilities,0.000000


In [13]:
#Booking
#Loại bỏ missing-value records
booking_df = booking_df.dropna()

In [14]:
booking_df=booking_df.reset_index(drop=True)

In [15]:
#Traveloka

#Thay '[]' bằng NULL
traveloka_df["Image"] = traveloka_df["Image"].apply(lambda y: np.nan if len(y)==2 else y)

percent_missing = traveloka_df.isnull().sum() * 100 / len(traveloka_df)
missing_value_df = pd.DataFrame({'column_name': traveloka_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
City,City,0.000000
Hotel name,Hotel name,0.000000
Image,Image,0.627240
Url,Url,0.000000
Address,Address,0.806452
Stars,Stars,0.000000
Min Price,Min Price,0.000000
Rating,Rating,0.000000
Number of reviews,Number of reviews,0.000000
Reviews,Reviews,0.000000


In [16]:
#Traveloka
traveloka_df = traveloka_df.dropna()

In [17]:
traveloka_df=traveloka_df.reset_index(drop=True)

In [18]:
#Mytour
missing_stars_hotel_url = mytour_df["Url"].loc[mytour_df["Stars"]==0]
len(missing_stars_hotel_url)

107

In [19]:
#Mytour
url_f = "/content/drive/MyDrive/20212/THDL/url_data/data/CHeck/Fill_url.csv"
fill_stars_hotel_url = pd.read_csv(url_f)
fill_stars_hotel_url.head()

,Url,Stars
0,https://mytour.vn/khach-san/972-khach-san-impe...,4
1,https://mytour.vn/khach-san/1486-khach-san-la-...,4
2,https://mytour.vn/khach-san/470-khach-san-inte...,5
3,https://mytour.vn/khach-san/971-khach-san-pull...,5
4,https://mytour.vn/khach-san/48189-khach-san-no...,5


In [20]:
#Mytour
for i in range(len(fill_stars_hotel_url)):
  url = fill_stars_hotel_url["Url"][i].replace('\n','') + '\r\n'
  #print(url)
  star_rep = int(fill_stars_hotel_url["Stars"][i])

  mytour_df["Stars"].loc[mytour_df["Url"]==url] = star_rep
  #print(mytour_df["Stars"].loc[mytour_df["Url"]==url])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [21]:
#Mytour
#Kiểm tra xem còn giá trị Star = 0 nào không
len(mytour_df.loc[mytour_df["Stars"]==0])

4

In [22]:
#Mytour
mytour_df = mytour_df.drop([210, 323, 921, 922])

In [23]:
#Mytour
mytour_df=mytour_df.reset_index(drop=True)

In [24]:
#Mytour
percent_missing = mytour_df.isnull().sum() * 100 / len(mytour_df)
missing_value_df = pd.DataFrame({'column_name': mytour_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
City,City,0.000000
Hotel name,Hotel name,0.309837
Image,Image,0.000000
Url,Url,0.000000
Address,Address,0.309837
Stars,Stars,0.000000
Min Price,Min Price,18.822618
Rating,Rating,0.309837
Number of reviews,Number of reviews,0.000000
Reviews,Reviews,0.000000


In [25]:
#Bởi vì không thể thêm các giá trị Min Price cho bản ghi bị missing-value, do đó ta sẽ xóa hết các bản ghi này
mytour_df = mytour_df.dropna(subset=["Min Price"])
mytour_df = mytour_df.reset_index(drop=True)

In [26]:
#Agoda
percent_missing = agoda_df.isnull().sum() * 100 / len(agoda_df)
missing_value_df = pd.DataFrame({'column_name': agoda_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
city,city,0.000000
hotel name,hotel name,0.376481
url,url,0.000000
address,address,31.192559
stars,stars,2.126010
price,price,3.033994
rating,rating,43.217805
location rating,location rating,88.329089
n_reviews,n_reviews,0.000000
image,image,11.947736


In [27]:
#Agoda
agoda_df = agoda_df.dropna()

In [28]:
#Agoda
agoda_df = agoda_df.reset_index(drop=True)

# **Chuẩn hóa dữ liệu**

**Price**

In [29]:
#Booking
booking_df['Price'] = booking_df['Price'].apply(lambda x: int(x.replace('VND', '').replace('.', '')))

In [30]:
#Traveloka
traveloka_df["Min Price"] = traveloka_df["Min Price"].apply(lambda x: x.replace('VND',''))
traveloka_df["Min Price"] = traveloka_df["Min Price"].apply(lambda x: int(x.replace('.','')))

In [31]:
#Mytour
mytour_df["Min Price"] = mytour_df["Min Price"].apply(lambda x: x.replace('đ',''))
mytour_df["Min Price"] = mytour_df["Min Price"].apply(lambda x: int(x.replace('.','')))

In [32]:
#Agoda
agoda_df['price'] = agoda_df['price'].apply(lambda x: str(x).replace('.', ''))

In [33]:
agoda_df['price'] = agoda_df['price'].apply(lambda x: int(x))

**Stars**

In [34]:
len(booking_df.loc[booking_df['Stars']>5])

0

In [35]:
len(traveloka_df.loc[traveloka_df['Stars']>5])

0

In [36]:
len(mytour_df.loc[mytour_df['Stars']>5])

0

In [37]:
len(agoda_df.loc[agoda_df['stars']>5])

223

In [38]:
#Agoda
agoda_df['stars'] = np.where(agoda_df['stars'] > 5, agoda_df['stars']/10 , agoda_df['stars'])

In [39]:
len(agoda_df.loc[agoda_df['stars']>5])

0

**Rating**

In [40]:
#Booking
booking_df['Rating'] = booking_df['Rating'].apply(lambda x: float(str(x).replace(',', '.')))

In [41]:
#Agoda
agoda_df['rating'] = agoda_df['rating'].apply(lambda x: float(str(x).replace(',', '.')))

**Number of Reviews**

In [42]:
#Booking
booking_df['Number of reviews'] = booking_df['Number of reviews'].apply(lambda x: re.sub('[^0-9]','', str(x)))

In [43]:
#Booking
booking_df['Number of reviews'] = booking_df['Number of reviews'].apply(lambda x: int(x))

In [44]:
#Agoda
agoda_df['n_reviews'] = agoda_df['n_reviews'].apply(lambda x: str(x).replace(' Nhận xét', ''))
agoda_df['n_reviews'] = agoda_df['n_reviews'].apply(lambda x: str(x).replace('.', ''))

In [45]:
#Agoda
agoda_df['n_reviews'] = agoda_df['n_reviews'].apply(lambda x: int(x))

In [46]:
#Agoda _ bỏ location rating
agoda_df = agoda_df.drop("location rating", 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


**Hotel name**

In [47]:
#Booking
booking_df['Hotel name'] = booking_df['Hotel name'].apply(lambda x: x.replace('\n', ' '))
booking_df['Hotel name'] = booking_df['Hotel name'].apply(lambda x: x.replace('\r', ''))

**City**

In [48]:
booking_df['City'].unique()

array(['Hà Nội', 'TP. Hồ Chí Minh', 'Đà Lạt', 'Đà Nẵng', 'Nha Trang',
       'Phú Quốc', 'Vũng Tàu', 'Hạ Long'], dtype=object)

In [49]:
traveloka_df['City'].unique()

array(['Hà Nội', 'Hồ Chí Minh', 'Đà Lạt', 'Đà Nẵng', 'Nha Trang',
       'Phú Quốc', 'Vũng Tàu', 'Hạ Long'], dtype=object)

In [50]:
mytour_df['City'].unique()

array(['Hà Nội', 'Hồ Chí Minh', 'Đà Lạt', 'Đà Nẵng', 'Nha Trang',
       'Phú Quốc', 'Vũng Tàu', 'Hạ Long'], dtype=object)

In [51]:
agoda_df['city'].unique()

array(['Nha Trang', 'Đảo Phú Quốc', 'Vũng Tàu', 'Ha Long',
       'Hồ Chí Minh', 'Hà Nội', 'Đà Nẵng', 'Đà Lạt'], dtype=object)

In [52]:
agoda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           981 non-null    object 
 1   hotel name     981 non-null    object 
 2   url            981 non-null    object 
 3   address        981 non-null    object 
 4   stars          981 non-null    float64
 5   price          981 non-null    int64  
 6   rating         981 non-null    float64
 7   n_reviews      981 non-null    int64  
 8   image          981 non-null    object 
 9   reviews        981 non-null    object 
 10  facilities     981 non-null    object 
 11  nearby places  981 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 92.1+ KB


In [53]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4875 entries, 0 to 4874
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               4875 non-null   object 
 1   Hotel name         4875 non-null   object 
 2   Url                4875 non-null   object 
 3   Address            4875 non-null   object 
 4   Stars              4875 non-null   int64  
 5   Price              4875 non-null   int64  
 6   Rating             4875 non-null   float64
 7   Number of reviews  4875 non-null   int64  
 8   Reviews            4875 non-null   object 
 9   Facilities         4875 non-null   object 
 10  Description        4875 non-null   object 
 11  Nearby places      4875 non-null   object 
 12  Image              4875 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 495.2+ KB


In [54]:
traveloka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1100 non-null   object 
 1   Hotel name         1100 non-null   object 
 2   Image              1100 non-null   object 
 3   Url                1100 non-null   object 
 4   Address            1100 non-null   object 
 5   Stars              1100 non-null   int64  
 6   Min Price          1100 non-null   int64  
 7   Rating             1100 non-null   float64
 8   Number of reviews  1100 non-null   int64  
 9   Reviews            1100 non-null   object 
 10  Facilities         1100 non-null   object 
 11  Description        1100 non-null   object 
 12  Nearby places      1100 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 111.8+ KB


In [55]:
mytour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1048 non-null   object 
 1   Hotel name         1048 non-null   object 
 2   Image              1048 non-null   object 
 3   Url                1048 non-null   object 
 4   Address            1048 non-null   object 
 5   Stars              1048 non-null   int64  
 6   Min Price          1048 non-null   int64  
 7   Rating             1048 non-null   float64
 8   Number of reviews  1048 non-null   int64  
 9   Reviews            1048 non-null   object 
 10  Facilities         1048 non-null   object 
 11  Description        1048 non-null   object 
 12  Nearby places      1048 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 106.6+ KB


# **Lọc trùng dữ liệu**

**Traveloka**

In [56]:
#Kiểm tra những Khách sạn trùng tên
dups_name = traveloka_df.pivot_table(columns=["Hotel name"], aggfunc='size')
print(dups_name.where(dups_name > 1).sum())

80.0


In [57]:
#Kiểm tra KS trùng địa chỉ và tên
dups_name_1 = traveloka_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
print(dups_name_1.where(dups_name_1 > 1).sum())

76.0


In [58]:
#Kiểm tra KS trùng địa chỉ, tên và giá
dups_name_p = traveloka_df.pivot_table(columns=["Hotel name", "Address", "Min Price"], aggfunc='size')
print(dups_name_p.where(dups_name_p > 1).sum())

74.0


In [59]:
#Lọc bỏ dữ liệu trùng lặp theo tên, địa chỉ và giá
traveloka_df = traveloka_df.drop_duplicates(subset=["Hotel name", "Address", "Min Price"])

In [60]:
dups_name_2 = traveloka_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
dups_name_2.loc[dups_name_2 > 1]

Hotel name                Address                                                                                                                          
Citadines Marina Hạ Long  Peninsula 3, Marina Halong,, Halong Marine Blvd, Hung Thang Ward, phường Hùng Thắng, Thành phố Hạ Long, Tỉnh Quảng Ninh, Việt Nam    2
dtype: int64

In [61]:
#2 KS này chung tỉnh và chỉ khác đôi chút ở địa chỉ --> Ta loại bỏ khách sạn có giá cao hơn
traveloka_df.loc[traveloka_df["Hotel name"] == "Citadines Marina Hạ Long"]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
1041,Hạ Long,Citadines Marina Hạ Long,['https://ik.imagekit.io/tvlk/apr-asset/TzEv3Z...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Peninsula 3, Marina Halong,, Halong Marine Blv...",4,2724384,0.0,0,[],"['Hồ bơi', 'Nhà hàng', 'Lễ tân 24h', 'WiFi', '...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Cái Dăm Market', '1.01 km'), ('Bệnh viện qu..."
1077,Hạ Long,Citadines Marina Hạ Long,['https://ik.imagekit.io/tvlk/apr-asset/TzEv3Z...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Peninsula 3, Marina Halong,, Halong Marine Blv...",4,4427121,0.0,0,[],"['Hồ bơi', 'Nhà hàng', 'Lễ tân 24h', 'WiFi', '...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Cái Dăm Market', '1.01 km'), ('Bệnh viện qu..."


In [62]:
traveloka_df = traveloka_df.drop([1077])

In [63]:
dups_name_2 = traveloka_df.pivot_table(columns=["Hotel name"], aggfunc='size')
dups_name_2.loc[dups_name_2 > 1]

Hotel name
Hoang Ha Hotel               2
Movenpick Resort Cam Ranh    2
dtype: int64

In [64]:
#2 KS này ở khác tỉnh nên có thể giữ lại
traveloka_df.loc[traveloka_df["Hotel name"] == "Hoang Ha Hotel"]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
292,Hồ Chí Minh,Hoang Ha Hotel,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"57 Trưong Quốc Dung, phường 10, Quận Phú Nhuận...",2,528000,8.1,43,"['', 'Khách sạn cũ, vệ sinh không tốt. Tuy nhi...","['Máy lạnh', 'Nhà hàng', 'Chỗ đậu xe', 'WiFi']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: 14:00...,"[('Jamiul Muslimin Mosque', '84 m'), ('Saigon ..."
501,Đà Lạt,Hoang Ha Hotel,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"203 Nguyễn Công Trứ, Phường 2, Đà Lạt, Tỉnh Lâ...",1,350000,0.0,4,"['cô chủ dễ thương, thuê xe rẻ với lại phòng ố...","['Chỗ đậu xe', 'Thang máy', 'WiFi']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: 14:00...,"[('Linh Son Pagoda', '702 m'), ('Big C Dalat',..."


In [65]:
#Tuy nhiên, 2 KS này chung tỉnh và chỉ khác đôi chút ở địa chỉ --> Ta loại bỏ khách sạn có giá cao hơn
traveloka_df.loc[traveloka_df["Hotel name"] == "Movenpick Resort Cam Ranh"]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
732,Nha Trang,Movenpick Resort Cam Ranh,['https://ik.imagekit.io/tvlk/apr-asset/TzEv3Z...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Plot D12, Cam Hai Dong, Cam Hải Đông, Nha Tran...",5,2896300,8.5,125,['Thời gian checkin khá lâu. Resort xếp khách ...,"['Hồ bơi', 'Nhà hàng', 'Lễ tân 24h', 'WiFi', '...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 15...,"[('Bãi Dài', '2.63 km'), ('Sân bay quốc tế C..."
741,Nha Trang,Movenpick Resort Cam Ranh,['https://ik.imagekit.io/tvlk/apr-asset/p8EuAB...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Plot D 12 ABC Cam Hai Dong CAM LAM DISTRIC, Ca...",5,3402786,0.0,2,"['Ở căn hộ condotel tiện lợi, có bếp tủ lạnh m...",[''],Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Gac Ma Vietnam Soldier Monument', '5.76 km'..."


In [66]:
traveloka_df = traveloka_df.drop([741])

In [67]:
#Lọc KS có trùng địa chỉ
dups_name_add = traveloka_df.pivot_table(columns=["Address"], aggfunc='size')
dups_name_add.where(dups_name_add > 1).sum()

6.0

In [68]:
traveloka_df.loc[traveloka_df['Address'].isin(["15 Nguyễn An Ninh, Côn Đảo (Hòn Sơn), Bà Rịa - Vũng Tàu, Việt Nam, 794317",
                                               "2 Nguyễn Thị Minh Khai, Lộc Thọ, Nha Trang, Khánh Hòa, Việt Nam, 650000",
                                               "Hồ Tràm, Xã Phước Thuận, Thành phố Vũng Tàu, Xuyên Mộc, Bà Rịa - Vũng Tàu, Việt Nam, 794978"])]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
735,Nha Trang,Panorama Central Nha Trang,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"2 Nguyễn Thị Minh Khai, Lộc Thọ, Nha Trang, Kh...",3,1126505,0.0,7,"['Tôi đã có kỳ nghỉ tuyệt vời tại đây', 'phòng...",[''],Giờ nhận và trả phòng\r\nGiờ nhận phòng: 14:00...,"[('Quảng trường 2 tháng 4', '201 m'), ('Nha Tr..."
739,Nha Trang,Handy Panorama Nha Trang Ocean View,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"2 Nguyễn Thị Minh Khai, Lộc Thọ, Nha Trang, Kh...",4,890000,8.7,32,"['', 'Giao dịch nhanh, giá rất tốt. Ngay trung...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Quảng trường 2 tháng 4', '236 m'), ('Nha Tr..."
918,Vũng Tàu,Ho Tram Beach Boutique Resort & Spa,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Hồ Tràm, Xã Phước Thuận, Thành phố Vũng Tàu, X...",4,5432400,8.7,164,"['Nhân viên thân thiện, bếp ăn ngon. Dịch vụ c...","['Máy lạnh', 'Nhà hàng', 'Hồ bơi', 'Lễ tân 24h...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 15...,"[('Bãi biển Hồ Tràm', '4.89 km'), ('Loc An Mar..."
1009,Vũng Tàu,Hoang Ngoc Hotel Con Dao,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"15 Nguyễn An Ninh, Côn Đảo (Hòn Sơn), Bà Rịa -...",1,588235,0.0,3,[],"['Máy lạnh', 'WiFi']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: 14:00...,"[('Trại giam Phú Hải', '261 m'), ('Sân bay Côn..."
1015,Vũng Tàu,Hong Ngoc Hotel,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"15 Nguyễn An Ninh, Côn Đảo (Hòn Sơn), Bà Rịa -...",1,529412,9.4,14,"['Phòng sạch, nhân viên thân thiện', 'Tot, gan...","['Máy lạnh', 'WiFi']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: 14:00...,"[('Trại giam Phú Hải', '242 m'), ('Sân bay Côn..."
1022,Vũng Tàu,Vietsovpetro Resort,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Hồ Tràm, Xã Phước Thuận, Thành phố Vũng Tàu, X...",4,7000000,7.8,18,"['', '', 'Gia đình tôi đã có 1 kỳ nghỉ lễ tuyệ...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Chỗ đậ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Loc An Market', '5.14 km'), ('Bãi biển Hồ T..."


In [69]:
#Ta có thể lọc bỏ bản ghi 739 và 1009
traveloka_df = traveloka_df.drop([739, 1009])

In [70]:
#Reset index
traveloka_df = traveloka_df.reset_index(drop=True)

**Mytour**

In [71]:
#Kiểm tra những Khách sạn trùng tên
dups_name = mytour_df.pivot_table(columns=["Hotel name"], aggfunc='size')
print(dups_name.where(dups_name > 1).sum())

28.0


In [72]:
#Kiểm tra KS trùng địa chỉ và tên
dups_name_1 = mytour_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
print(dups_name_1.where(dups_name_1 > 1).sum())

26.0


In [73]:
#Kiểm tra KS trùng địa chỉ, tên và giá
dups_name_p = mytour_df.pivot_table(columns=["Hotel name", "Address", "Min Price"], aggfunc='size')
print(dups_name_p.where(dups_name_p > 1).sum())

26.0


In [74]:
#Lọc bỏ dữ liệu trùng lặp theo tên, địa chỉ và giá
mytour_df = mytour_df.drop_duplicates(subset=["Hotel name", "Address", "Min Price"])

In [75]:
dups_name_2 = mytour_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
print(dups_name_2.where(dups_name_2 > 1).sum())

0.0


In [76]:
dups_name_3 = mytour_df.pivot_table(columns=["Hotel name"], aggfunc='size')
print(dups_name_3.where(dups_name_3 > 1).sum())

2.0


In [77]:
dups_name_3.loc[dups_name_3 > 1]

Hotel name
Khách Sạn Galaxy    2
dtype: int64

In [78]:
#Do 2 KS này có thể là do cùng chi nhánh ở TP.Hồ Chí Minh --> Ta bỏ KS có giá cao hơn
mytour_df.loc[mytour_df["Hotel name"] == "Khách Sạn Galaxy"]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
335,Hồ Chí Minh,Khách Sạn Galaxy,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/23480-khach-san-ga...,"10-12-14 Đường Nội Khu Hưng Gia 3, Tân Phong, ...",2,629030,10.0,1,"[""Excellent value at an extremely friendly hot...","['Cho thuê xe máy', 'Đưa/đón khách sân bay', '...","['Tọa lạc tại Thành phố Hồ Chí Minh, nằm trong...","[('Red&Round', '41 m'), ('Cafe De Bodyfriend',..."
375,Hồ Chí Minh,Khách Sạn Galaxy,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/46922-khach-san-ga...,"38/6, Lam Sơn, Phường 2, Quận Tân Bình, Hồ Chí...",2,493546,7.0,0,[],"['Đưa/đón khách sân bay', 'Giữ hành lý', 'Báo ...","['Cách sân bay Tân Sơn Nhất chỉ 700m, với 10 p...","[('Chùa Phổ Quang', '750 m'), ('Bảo Tàng Không..."


In [79]:
mytour_df = mytour_df.drop([335])

In [80]:
#Lọc KS có trùng địa chỉ
dups_name_add = mytour_df.pivot_table(columns=["Address"], aggfunc='size')
dups_name_add.where(dups_name_add > 1).sum()

13.0

In [81]:
dups_name_add.loc[dups_name_add > 1]

Address
., Đảo Hòn Tre, Thành Phố Nha Trang, Khánh Hòa, Việt Nam           2
3/1 , Trần Quang Khải, Thành Phố Nha Trang, Khánh Hòa, Việt Nam    2
6b, Láng Hạ, Quận Ba Đình, Hà Nội, Việt Nam                        2
86/4 , Trần Phú, Thành Phố Nha Trang, Khánh Hòa, Việt Nam          2
Khu Bãi Dài, Gành Dầu, Phú Quốc, Kiên Giang, Việt Nam              2
Tổ 5, Ấp Ông Lang, Cửa Dương, Phú Quốc, Kiên Giang, Việt Nam       3
dtype: int64

In [82]:
mytour_df.loc[mytour_df['Address'].isin(["., Đảo Hòn Tre, Thành Phố Nha Trang, Khánh Hòa, Việt Nam", 
                                         "3/1 , Trần Quang Khải, Thành Phố Nha Trang, Khánh Hòa, Việt Nam", 
                                         "6b, Láng Hạ, Quận Ba Đình, Hà Nội, Việt Nam", 
                                         "86/4 , Trần Phú, Thành Phố Nha Trang, Khánh Hòa, Việt Nam", 
                                         "Khu Bãi Dài, Gành Dầu, Phú Quốc, Kiên Giang, Việt Nam", 
                                         "Tổ 5, Ấp Ông Lang, Cửa Dương, Phú Quốc, Kiên Giang, Việt Nam"])]

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
30,Hà Nội,Khách Sạn Fortuna Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/1377-khach-san-for...,"6b, Láng Hạ, Quận Ba Đình, Hà Nội, Việt Nam",4,1239609,8.0,172,"['Thất vọng', 'Rất tốt\nLocation,B1', 'Kém', '...","['Lễ tân 24h', 'Giữ hành lý', 'Internet miễn p...",['Khách sạn Fortuna là một trong những cơ sở l...,"[('Hakata Kouryu', '69 m'), ('May Mắn Restaura..."
59,Hà Nội,Khách Sạn The Pearl Hanoi,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/4747-khach-san-the...,"6b, Láng Hạ, Quận Ba Đình, Hà Nội, Việt Nam",3,756770,7.8,49,"['Nhân viên nhiệt tình, chú đáo với khách\nDo ...","['Lễ tân 24h', 'Thu đổi ngoại tệ', 'Bán tour',...","['Được xây dựng vào tháng 2 năm 2013, Khách Sạ...","[('May Mắn Restaurant', '28 m'), ('Nhà Hàng Th..."
805,Nha Trang,Vinpearl Luxury Nha Trang,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/498-vinpearl-luxur...,"., Đảo Hòn Tre, Thành Phố Nha Trang, Khánh Hòa...",5,3565715,8.8,124,['Khách sạn nhìn chung là ổn\nKhách sạn nằm ng...,"['Đưa/đón khách sân bay', 'Phòng gym', 'Đặt vé...",['Vinpearl Luxury Nha Trang đem đến một nơi tr...,"[('VietStreet - Món Ăn Đường Phố', '3.4 km'), ..."
808,Nha Trang,Khách Sạn Dendro Gold Nha Trang,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/23714-khach-san-de...,"86/4 , Trần Phú, Thành Phố Nha Trang, Khánh Hò...",4,475326,8.8,28,"['A happy place, hài lòng\nĐiểm 10 về sự nhiệt...","['Đưa/đón khách sân bay', 'Phòng gym', 'Đặt vé...",['Khách Sạn Dendro Gold Nha Trang có không gia...,"[('Nhà Bar', '26 m'), ('Dinner at Cim’s', '124..."
815,Nha Trang,Vinpearl Discovery Golflink Nha Trang,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/48305-vinpearl-dis...,"., Đảo Hòn Tre, Thành Phố Nha Trang, Khánh Hòa...",5,10348053,7.0,0,[],"['Đưa/đón khách sân bay', 'Đưa/đón khách bến p...",['Vinpearl Discovery Golflink Nha Trang (Vinpe...,"[('Bella Ciao Club Nha Trang', '6.9 km')]"
830,Nha Trang,Khách Sạn Golden Holiday Nha Trang,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/2235-khach-san-gol...,"3/1 , Trần Quang Khải, Thành Phố Nha Trang, Kh...",3,528140,8.0,249,"['Trải nghiệm tuyệt vời\nKhách sạn sạch sẽ,giá...","['Bán tour', 'Lễ tân 24h', 'Giữ hành lý', 'Int...",['Khách sạn Golden Holiday là điểm du lịch bổ ...,"[('Quán Hẻm Korean Restaurant', '44 m'), ('Nhà..."
844,Nha Trang,Khách sạn Rigel Nha Trang,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/38389-khach-san-ri...,"86/4 , Trần Phú, Thành Phố Nha Trang, Khánh Hò...",4,454200,9.2,122,"['Rigel hotel Thang 6/2022\nPhòng sạch, nhân v...","['American Express', 'Đưa/đón khách sân bay', ...","['Tọa lạc tại thành phố Nha Trang, Rigel Hotel...","[('Nhà Bar', '162 m'), ('Hybrid', '206 m'), ('..."
865,Nha Trang,Khách Sạn Golden Rain 2,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/9824-khach-san-gol...,"3/1 , Trần Quang Khải, Thành Phố Nha Trang, Kh...",3,260276,7.8,28,['- Vị trí: Ngay khu trung tâm nên tiện lợi ch...,"['Đưa/đón khách sân bay', 'Bán tour', 'Két an ...","['Nằm ở vị trí thuận lợi trong Lộc Thọ, Golden...",[]
883,Phú Quốc,Kingo Retreat Resort Phú Quốc,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/49434-kingo-retrea...,"Tổ 5, Ấp Ông Lang, Cửa Dương, Phú Quốc, Kiên G...",4,939053,9.0,41,"['Du lịch các gia đình.\nDịch vụ ở đây ok,nhân...","['Lễ tân 24h', 'Giữ hành lý', 'Bán tour', 'Int...",['Kingo Retreat Resort Phú Quốc cách bãi biển ...,"[('Coco Palm Restaurant', '405 m'), ('Nhà hàng..."
884,Phú Quốc,Vinpearl Resort & Golf Phú Quốc,['https://storage.googleapis.com/tripi-assets/...,https://mytour.vn/khach-san/23193-vinpearl-res...,"Khu Bãi Dài, Gành Dầu, Phú Quốc, Kiên Giang, V...",5,2874546,8.8,191,['Đánh giá Vin resort and go

In [83]:
mytour_df = mytour_df.drop([30, 808, 815, 830])

In [84]:
mytour_df = mytour_df.reset_index(drop=True)

**Agoda**

In [85]:
#Kiểm tra những Khách sạn trùng tên
ago_name = agoda_df.pivot_table(columns=["hotel name"], aggfunc='size')
print(ago_name.where(ago_name > 1).sum())

323.0


In [86]:
#Kiểm tra KS trùng địa chỉ và tên
ago_name_1 = agoda_df.pivot_table(columns=["hotel name", "address"], aggfunc='size')
print(ago_name_1.where(ago_name_1 > 1).sum())

323.0


In [87]:
#Kiểm tra KS trùng địa chỉ, tên và giá
ago_name_p = agoda_df.pivot_table(columns=["hotel name", "address", "price"], aggfunc='size')
print(ago_name_p.where(ago_name_p > 1).sum())

323.0


In [88]:
#Lọc bỏ dữ liệu trùng lặp theo tên, địa chỉ và giá
agoda_df = agoda_df.drop_duplicates(subset=["hotel name", "address", "price"])

In [89]:
#Check
ago_name_2 = agoda_df.pivot_table(columns=["hotel name", "address"], aggfunc='size')
print(ago_name_2.where(ago_name_2 > 1).sum())

0.0


In [90]:
#Check
ago_name_3 = agoda_df.pivot_table(columns=["hotel name"], aggfunc='size')
print(ago_name_3.where(ago_name_3 > 1).sum())

0.0


In [91]:
#Check
ago_name_3 = agoda_df.pivot_table(columns=["address"], aggfunc='size')
print(ago_name_3.where(ago_name_3 > 1).sum())

17.0


In [92]:
ago_name_3.loc[ago_name_3 > 1]

address
16 Tôn Đản, Lộc Thọ, Nha Trang, Việt Nam    2
Cảng Hòn Gai, Ha Long, Việt Nam             2
Lộc Thọ, Nha Trang, Việt Nam                4
Phước Mỹ, Đà Nẵng, Việt Nam                 3
Quận 7, Hồ Chí Minh, Việt Nam               2
Đảo Tuần Châu, Ha Long, Việt Nam            4
dtype: int64

In [93]:
agoda_df.loc[agoda_df['address'].isin(["16 Tôn Đản, Lộc Thọ, Nha Trang, Việt Nam", 
                                      "Cảng Hòn Gai, Ha Long, Việt Nam", 
                                      "Lộc Thọ, Nha Trang, Việt Nam", 
                                      "Phước Mỹ, Đà Nẵng, Việt Nam", 
                                      "Quận 7, Hồ Chí Minh, Việt Nam", 
                                      "Đảo Tuần Châu, Ha Long, Việt Nam"])]

,city,hotel name,url,address,stars,price,rating,n_reviews,image,reviews,facilities,nearby places
29,Nha Trang,Khách Sạn & Căn Hộ Maple (Maple Hotel & Apartm...,https://www.agoda.com/vi-vn/maple-hotel-apartm...,"16 Tôn Đản, Lộc Thọ, Nha Trang, Việt Nam",4.5,362216,8.6,1923,https://pix6.agoda.net/hotelImages/2537789/-1/...,[],"['Nằm ở trung tâm Nha Trang', 'Dọn phòng hằng ...",['Bảo tàng Hải dương học quốc gia Việt Nam - 3...
40,Nha Trang,Sea View Luxury Zoom Apartment,https://www.agoda.com/vi-vn/sea-view-luxury-zo...,"Lộc Thọ, Nha Trang, Việt Nam",4.5,1246753,8.4,147,https://pix6.agoda.net/hotelImages/163/1638950...,[],"['Điều hòa', 'Đưa đón sân bay', 'Wi-Fi miễn ph...","['Hon Chong - 3,36 km', 'i resort - 3,79 km', ..."
51,Nha Trang,Sunrise Ocean View Apartment,https://www.agoda.com/vi-vn/sunrise-ocean-view...,"Lộc Thọ, Nha Trang, Việt Nam",4.5,337662,8.3,659,https://pix6.agoda.net/hotelImages/985079/-1/8...,['Tôi ở căn 1 phòng ngủ 50m /tối đa 4 người lớ...,"['Nằm ở trung tâm Nha Trang', 'Đưa đón sân bay...",['Bảo tàng Hải dương học quốc gia Việt Nam - 3...
62,Nha Trang,Diamond Bay Residence,https://www.agoda.com/vi-vn/diamond-bay-reside...,"Lộc Thọ, Nha Trang, Việt Nam",4.5,788961,8.1,274,https://pix6.agoda.net/hotelImages/568/568160/...,[],"['Đưa đón sân bay', 'Wi-Fi miễn phí trong tất ...","['Hon Chong - 3,01 km', 'i resort - 3,5 km', '..."
73,Nha Trang,Căn hộ Galaxy (Galaxy Apartment),https://www.agoda.com/vi-vn/galaxy-apartment/h...,"Lộc Thọ, Nha Trang, Việt Nam",4.0,182638,7.9,581,https://pix6.agoda.net/hotelImages/625/625297/...,[],"['Điều hòa', 'Đưa đón sân bay', 'Cách bãi biển...","['Hon Chong - 3,95 km', 'i resort - 4,1 km', '..."
107,Nha Trang,Khách sạn Pavillon Garden Nha Trang (Pavillon ...,https://www.agoda.com/vi-vn/pavillon-garden-ho...,"16 Tôn Đản, Lộc Thọ, Nha Trang, Việt Nam",3.0,1594371,7.8,760,https://pix5.agoda.net/hotelImages/699/699308/...,['Có điểm trừ là phòng hơi nhỏ và ngoài ban cô...,"['Nằm ở trung tâm Nha Trang', 'Dọn phòng hằng ...",['Bảo tàng Hải dương học quốc gia Việt Nam - 3...
211,Ha Long,Du Thuyền Paloma Cruise Hạ Long (Halong Paloma...,https://www.agoda.com/vi-vn/halong-paloma-crui...,"Cảng Hòn Gai, Ha Long, Việt Nam",4.0,6493506,8.9,745,https://pix6.agoda.net/hotelImages/247936/-1/5...,[],"['Đưa đón sân bay', 'Wi-Fi miễn phí trong tất ...","['Đảo Soi Sim - 11,64 km', 'Hang Luồn - 12,93 ..."
214,Ha Long,Paradise Elegance Cruise Halong,https://www.agoda.com/vi-vn/paradise-elegance-...,"Đảo Tuần Châu, Ha Long, Việt Nam",5.0,5863636,8.9,210,https://pix5.agoda.net/hotelImages/116/1160092...,[],"['Dọn phòng hằng ngày', 'Điều hòa', 'Đưa đón s...","['Đảo Soi Sim - 12,55 km', 'Hang Luồn - 20,65 ..."
215,Ha Long,Halong Legacy Cruise,https://www.agoda.com/vi-vn/halong-legacy-crui...,"Đảo Tuần Châu, Ha Long, Việt Nam",3.0,7913043,8.3,480,https://pix5.agoda.net/hotelImages/523292/-1/0...,[],"['Quán bar', 'Điều hòa', 'Đưa đón sân bay', 'R...","['Đảo Soi Sim - 9,53 km', 'Hang Luồn - 21,65 k..."
218,Ha Long,Alisa Premier Cruise,https://www.agoda.com/vi-vn/alisa-premier-crui...,"Đảo Tuần Châu, Ha Long, Việt Nam",5.0,5840869,9.2,743,https://pix6.agoda.net/hotelImages/2537815/-1/...,[],"['Đưa đón sân bay', 'Wi-Fi miễn phí trong tất ...","['Đảo Soi Sim - 11,68 km', 'Hang Luồn - 20,4 k..."


In [94]:
agoda_df = agoda_df.drop([322])

In [95]:
agoda_df = agoda_df.reset_index(drop=True)

**Booking**

In [96]:
#Kiểm tra những Khách sạn trùng tên
dups_name = booking_df.pivot_table(columns=["Hotel name"], aggfunc='size')
#Có 101 KS trùng tên
dups_name.where(dups_name > 1).sum()

101.0

In [97]:
#Kiểm tra KS trùng địa chỉ và tên
dups_name_1 = booking_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
#Có 45 KS trùng địa chỉ và tên
dups_name_1.where(dups_name_1 > 1).sum()

45.0

In [98]:
#Kiểm tra KS trùng địa chỉ, tên và giá
dups_name_p = booking_df.pivot_table(columns=["Hotel name", "Address", "Price"], aggfunc='size')
#Có 32 KS trùng địa chỉ, tên và giá
dups_name_p.where(dups_name_p > 1).sum()

32.0

In [99]:
#Lọc bỏ dữ liệu trùng lặp theo tên, địa chỉ và giá
booking_df = booking_df.drop_duplicates(subset=["Hotel name", "Address", "Price"])

In [100]:
#Check
dups_name_2 = booking_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
#Như vậy, sau khi loại bước đầu, còn 13 KS trùng tên và địa chỉ
dups_name_2.where(dups_name_2 > 1).sum()

13.0

In [101]:
dups_name_2.loc[dups_name_2 > 1]

Hotel name                                                             Address                                                        
Căn hộ Melody Vũng Tàu                                                 149 Võ Thị Sáu, Vũng Tàu, Việt Nam                              2
Căn hộ Mới trên Booking.com Căn Hộ 2 Phòng Ngủ Mường Thanh Viễn Triều  3 Phạm Văn Đồng, Nha Trang, Việt Nam                              2
Căn hộ Mới trên Booking.com Happy stay home                            281/27/1 Lê Văn Sỹ, Quận Tân Bình, TP. Hồ Chí Minh, Việt Nam    3
Căn hộ Mới trên Booking.com LASON Nha Trang Hotel & Apartment          Hoàng Tăng Bí, Nha Trang, Việt Nam                                2
Khách sạn HANZ Bao Hoang Hotel Dalat                                   48 Đường Hà Huy Tập, Đà Lạt, Việt Nam                             2
Khách sạn Hương Trà Hotel apartment                                    128 Hồ Nghinh, Đà Nẵng, Việt Nam                                  2
dtype: int64

In [102]:
ls_name = ["Căn hộ Melody Vũng Tàu",
           "Căn hộ Mới trên Booking.com Căn Hộ 2 Phòng Ngủ Mường Thanh Viễn Triều",
           "Căn hộ Mới trên Booking.com Happy stay home",
           "Căn hộ Mới trên Booking.com LASON Nha Trang Hotel & Apartment",
           "Khách sạn HANZ Bao Hoang Hotel Dalat",
           "Khách sạn Hương Trà Hotel apartment"]

In [103]:
#Do những KS này có thể là do cùng chi nhánh ở tỉnh nào --> Ta bỏ KS có giá cao hơn (1792, 1898, 2180, 2688, 3697, 3764, 4663)
booking_df.loc[booking_df["Hotel name"].isin(ls_name)]

,City,Hotel name,Url,Address,Stars,Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places,Image
1792,TP. Hồ Chí Minh,Căn hộ Mới trên Booking.com Happy stay home,https://www.booking.com/hotel/vn/happy-stay-ho...,"281/27/1 Lê Văn Sỹ, Quận Tân Bình, TP. Hồ Chi...",0,201600,0.0,0,[],"['Cho phép mang theo vật nuôi', 'Điều hòa nh...",Happy stay home is situated in the Tan Binh di...,"[('Sân vận động Quân khu 7', '0,9 km'), ('Đại ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1892,TP. Hồ Chí Minh,Căn hộ Mới trên Booking.com Happy stay home,https://www.booking.com/hotel/vn/happy-stay-ho...,"281/27/1 Lê Văn Sỹ, Quận Tân Bình, TP. Hồ Chi...",0,192000,0.0,0,[],"['Chỗ đỗ xe miễn phí', 'Cho phép mang the...",Happy stay home is situated in the Tan Binh di...,"[('Sân vận động Quân khu 7', '0,9 km'), ('Đại ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1898,TP. Hồ Chí Minh,Căn hộ Mới trên Booking.com Happy stay home,https://www.booking.com/hotel/vn/happy-stay-ho...,"281/27/1 Lê Văn Sỹ, Quận Tân Bình, TP. Hồ Chi...",0,250000,0.0,0,[],"['Chỗ đỗ xe', 'Điều hòa nhiệt độ']",Situated in Ho Chi Minh City in the Ho Chi Min...,"[('Sân vận động Quân khu 7', '0,9 km'), ('Đại ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2061,Đà Lạt,Khách sạn HANZ Bao Hoang Hotel Dalat,https://www.booking.com/hotel/vn/hanz-bao-hong...,"48 Đường Hà Huy Tập, Đà Lạt, Việt Nam",0,271188,8.0,2,[],"['Wi-Fi miễn phí', 'Chỗ đỗ xe', 'Cho phép...",Set in Da Lat and with Lam Vien Square reachab...,"[('Cock Church', '0,4 km'), ('Lam Vien Square'...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2180,Đà Lạt,Khách sạn HANZ Bao Hoang Hotel Dalat,https://www.booking.com/hotel/vn/bao-dang.vi.h...,"48 Đường Hà Huy Tập, Đà Lạt, Việt Nam",0,371700,8.3,43,"['“Phòng ốc rộng rãi, thoáng! Nhân viên Thân t...","['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Cock Church', '0,4 km'), ('Lam Vien Square'...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2688,Đà Nẵng,Khách sạn Hương Trà Hotel apartment,https://www.booking.com/hotel/vn/huong-tra-apa...,"128 Hồ Nghinh, Đà Nẵng, Việt Nam",1,300000,9.0,1,[],"['Wi-Fi miễn phí', 'Giáp biển', 'Dịch vụ p...",Chỗ nghỉ này cách bãi biển 3 phút đi bộ. Set i...,"[('Làng chài Mân Thái', '1,6 km'), ('Cầu sông ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
3193,Đà Nẵng,Khách sạn Hương Trà Hotel apartment,https://www.booking.com/hotel/vn/huong-tra-apa...,"128 Hồ Nghinh, Đà Nẵng, Việt Nam",2,270000,7.0,1,[],"['Xe đưa đón sân bay', 'Giáp biển', 'Phòng g...",Chỗ nghỉ này cách bãi biển 9 phút đi bộ. Hương...,"[('Làng chài Mân Thái', '1,6 km'), ('Cầu sông ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
3406,Nha Trang,Căn hộ Mới trên Booking.com Căn Hộ 2 Phòng Ngủ...,https://www.booking.com/hotel/vn/can-ho-2-phon...,"3 Phạm Văn Đồng, Nha Trang, Việt Nam",0,700000,0.0,0,[],"['Xe đưa đón sân bay', 'Giáp biển', 'Trung tâ...",Chỗ nghỉ này cách bãi biển 1 phút đi bộ. Căn H...,"[('Hòn Chồng', '0,5 km'), ('Chùa Hòn Đỏ', '1,1...",['https://cf.bstatic.com/xdata/images/hotel/ma...
3697,Nha Trang,Căn hộ Mới trên Booking.com LASON Nha Trang Ho...,https://www.booking.com/hotel/vn/lason-nha-tra...,"Hoàng Tăng Bí, Nha Trang, Việt Nam",0,360000,0.0,0,[],"['Phòng không hút thuốc', 'Giáp biển', 'Chô...",Chỗ nghỉ này cách bãi biển 12 phút đi bộ. 1.3 ...,"[('Hòn Chồng', '2 km'), ('Chùa Hòn Đỏ', '2,7 k...",['https://cf.bstatic.com/xdata/images/hotel/ma...
3754,Nha Trang,Căn hộ Mới trên Booking.com LASON Nha Trang Ho...,https://www.booking.com/hotel/vn/lason-nha-tra...,"Hoàng Tăng Bí, Nha Trang, Việt Nam",0,300000,0.0,0,[],"['Phòng không hút thuốc', 'Phòng gia đình...",Chỗ nghỉ này cách bãi biển 12 phút đi bộ. LASO...,"[('Hòn Chồng', '2 km'), ('Chùa Hòn Đỏ', '2,7 k...",['https://cf.bstatic.com/xdata/images/hotel/ma...


In [104]:
booking_df = booking_df.drop([1792, 1898, 2180, 2688, 3697, 3764, 4662])

In [105]:
#Done _ (1)
sdups_name_2 = booking_df.pivot_table(columns=["Hotel name", "Address"], aggfunc='size')
sdups_name_2.where(sdups_name_2 > 1).sum()

0.0

In [106]:
#Check
dups_name_3 = booking_df.pivot_table(columns=["Hotel name"], aggfunc='size')
#Vẫn còn 56 KS trùng tên
dups_name_3.where(dups_name_3 > 1).sum()

56.0

In [107]:
#Có thể những KS này là chi nhánh trong cùng 1 thành phố --> bỏ những KS giá cao (615, )
#Nếu khác thành phố thì giữ lại
dups_name_4 = booking_df.pivot_table(columns=["Hotel name", "City"], aggfunc='size')
#Vẫn còn 15 KS trùng tên
dups_name_4.where(dups_name_4 > 1).sum()

15.0

In [108]:
dups_name_4.loc[dups_name_4 > 1]

Hotel name                                                    City             
Biệt thự Mới trên Booking.com Kẻ Mơ's House                   Đà Lạt               2
Biệt thự Sky Villa                                            Vũng Tàu           2
Căn hộ Langmandi Apartment                                    Hà Nội             3
Căn hộ Tiến Đạt Apartment                                     Nha Trang            2
Khách sạn Galaxy Hotel                                        TP. Hồ Chí Minh    2
Khách sạn Royal Hotel                                         Hà Nội             2
Nhà nghỉ mát Mới trên Booking.com Artichoke Home & Apartment  Đà Lạt               2
dtype: int64

In [109]:
ls_name_1 = ["Biệt thự Mới trên Booking.com Kẻ Mơ's House",                     
            "Biệt thự Sky Villa",                                                
            "Căn hộ Langmandi Apartment",                                                                            
            "Căn hộ Tiến Đạt Apartment",                                         
            "Khách sạn Galaxy Hotel",                                                
            "Khách sạn Royal Hotel",                                       
            "Nhà nghỉ mát Mới trên Booking.com Artichoke Home & Apartment",                                          
             ]

In [110]:
#Lọc bỏ (315, 615, 735, 1555, 2250, 2582, 3482, 4382)
booking_df.loc[booking_df["Hotel name"].isin(ls_name_1)]

,City,Hotel name,Url,Address,Stars,Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places,Image
315,Hà Nội,Khách sạn Royal Hotel,https://www.booking.com/hotel/vn/royal-ba-dinh...,"62 Phạm Huy Thông Ngọc Khánh , Ba Đình , Hanoi...",3,560000,9.4,227,"['“Vị trí đẹp, view đẹp, lân cận có nhiều tiện...","['1 hồ bơi', 'Xe đưa đón sân bay', 'Phòng kh...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Hồ Ngọc Khánh', '0,2 km'), ('Đại học RMIT',...",['https://cf.bstatic.com/xdata/images/hotel/ma...
566,Hà Nội,Căn hộ Langmandi Apartment,https://www.booking.com/hotel/vn/langmandi-apa...,"58 Ấu Triệu, Quận Hoàn Kiếm, Hà Nội, Việt Nam",3,437400,7.7,45,"['“bếp đầy đủ tiện nghi, giường thoải mái, vị ...","['Phòng không hút thuốc', 'Wi-Fi miễn phí...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Nhà thờ Thánh Joseph', '0,1 km'), ('Nationa...",['https://cf.bstatic.com/xdata/images/hotel/ma...
579,Hà Nội,Khách sạn Royal Hotel,https://www.booking.com/hotel/vn/royal-hanoi1....,"19 Hang Chao Street, Cat Linh Ward, Dong Da Di...",3,549400,6.8,58,[],"['Xe đưa đón sân bay', 'Phòng không hút thu...","Nằm ở thành phố Hà Nội, cách Văn Miếu - Quốc T...","[('Văn Miếu - Quốc Tử Giám', '0,2 km'), ('Sân ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
615,Hà Nội,Căn hộ Langmandi Apartment,https://www.booking.com/hotel/vn/langmandi-apa...,"51 Lê Hồng Phong, Quận Ba Đình, Hà Nội, Việ...",3,647190,7.8,14,[],"['Phòng không hút thuốc', 'Phòng gia đình...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Chùa Một Cột', '0,2 km'), ('Lăng Chủ tịch H...",['https://cf.bstatic.com/xdata/images/hotel/ma...
735,Hà Nội,Căn hộ Langmandi Apartment,https://www.booking.com/hotel/vn/langmandi-apa...,"8 Nhà Thờ, Quận Hoàn Kiếm, Hà Nội, Việt Nam",0,575280,5.0,1,[],"['Phòng không hút thuốc', 'Phòng gia đình...","Located in the centre of Hanoi, 200 metres fro...","[('Nhà thờ Thánh Joseph', '0,2 km'), ('Hồ Hoàn...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1550,TP. Hồ Chí Minh,Khách sạn Galaxy Hotel,https://www.booking.com/hotel/vn/galaxy-go-vap...,594-596 Nguyễn Oanh phường 6 quận gò vấp 594-5...,0,405000,8.0,261,"['“Phòng sạch sẽ, thoáng mát, nv dễ thương, nh...","['Phòng không hút thuốc', 'Wi-Fi miễn phí...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Saigon Mall Go Vap', '2 km'), ('Vincom Plaz...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1555,TP. Hồ Chí Minh,Khách sạn Galaxy Hotel,https://www.booking.com/hotel/vn/galaxy-apartm...,"Số 12 đường nội khu Hưng Gia 3, Khu đô thị Phú...",0,1031200,8.6,27,['“Lễ tân rất dễ mến. Lần sau mình sẽ tiếp tục...,"['Phòng không hút thuốc', 'Wi-Fi miễn phí...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('SC VivoCity', '0,8 km'), ('Bệnh viện FV', '...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2250,Đà Lạt,Biệt thự Mới trên Booking.com Kẻ Mơ's House,https://www.booking.com/hotel/vn/ke-mos-house-...,"5 Đường Trần Khánh Dư, Đà Lạt, Việt Nam",0,3750000,0.0,0,[],"['Chỗ đỗ xe miễn phí', 'Cho phép mang the...","Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 1...","[('Dalat Palace Golf Club', '0,7 km'), ('Linh ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2540,Đà Lạt,Biệt thự Mới trên Booking.com Kẻ Mơ's House,https://www.booking.com/hotel/vn/ke-mos-house....,"9 Đường Trần Khánh Dư, Đà Lạt, Việt Nam",0,3150000,0.0,0,[],"['Chỗ đỗ xe miễn phí', 'Cho phép mang the...","Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 1...","[('Dalat Palace Golf Club', '0,7 km'), ('Linh ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2556,Đà Lạt,Nhà nghỉ mát Mới trên Booking.com Artichoke Ho...,https://www.booking.com/hotel/vn/artichoke-hom...,"8 Đường Thiên Thành, phường 4, Đà Lạt, Đà Lạt,...",0,945000,0.0,0,[],['Chỗ đỗ xe miễn phí'],Artichoke Home & Apartment is located in the X...,"[('Khách sạn Hằng Nga', '0,6 km'), ('Chùa Lâm ...",['https://cf.bstatic.com/xdata/images/hotel/ma...


In [111]:
booking_df = booking_df.drop([315, 615, 735, 1555, 2250, 2582, 3482, 4381])

In [112]:
#Check
sdups_name_4 = booking_df.pivot_table(columns=["Hotel name", "City"], aggfunc='size')
#Đã không còn KS trùng lặp
sdups_name_4.where(sdups_name_4 > 1).sum()

0.0

In [114]:
booking_df = booking_df.reset_index(drop=True)

In [115]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4844 entries, 0 to 4843
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               4844 non-null   object 
 1   Hotel name         4844 non-null   object 
 2   Url                4844 non-null   object 
 3   Address            4844 non-null   object 
 4   Stars              4844 non-null   int64  
 5   Price              4844 non-null   int64  
 6   Rating             4844 non-null   float64
 7   Number of reviews  4844 non-null   int64  
 8   Reviews            4844 non-null   object 
 9   Facilities         4844 non-null   object 
 10  Description        4844 non-null   object 
 11  Nearby places      4844 non-null   object 
 12  Image              4844 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 492.1+ KB


In [116]:
traveloka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1059 non-null   object 
 1   Hotel name         1059 non-null   object 
 2   Image              1059 non-null   object 
 3   Url                1059 non-null   object 
 4   Address            1059 non-null   object 
 5   Stars              1059 non-null   int64  
 6   Min Price          1059 non-null   int64  
 7   Rating             1059 non-null   float64
 8   Number of reviews  1059 non-null   int64  
 9   Reviews            1059 non-null   object 
 10  Facilities         1059 non-null   object 
 11  Description        1059 non-null   object 
 12  Nearby places      1059 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 107.7+ KB


In [117]:
mytour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1030 non-null   object 
 1   Hotel name         1030 non-null   object 
 2   Image              1030 non-null   object 
 3   Url                1030 non-null   object 
 4   Address            1030 non-null   object 
 5   Stars              1030 non-null   int64  
 6   Min Price          1030 non-null   int64  
 7   Rating             1030 non-null   float64
 8   Number of reviews  1030 non-null   int64  
 9   Reviews            1030 non-null   object 
 10  Facilities         1030 non-null   object 
 11  Description        1030 non-null   object 
 12  Nearby places      1030 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 104.7+ KB


In [118]:
agoda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           806 non-null    object 
 1   hotel name     806 non-null    object 
 2   url            806 non-null    object 
 3   address        806 non-null    object 
 4   stars          806 non-null    float64
 5   price          806 non-null    int64  
 6   rating         806 non-null    float64
 7   n_reviews      806 non-null    int64  
 8   image          806 non-null    object 
 9   reviews        806 non-null    object 
 10  facilities     806 non-null    object 
 11  nearby places  806 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 75.7+ KB


# **Tiền xử lý dữ liệu matching**

**Hotel name**

In [119]:
#Bước 1: Chuyển tên KS về dạng chữ thường
traveloka_df['new_name'] = traveloka_df['Hotel name'].str.lower()
mytour_df['new_name'] = mytour_df['Hotel name'].str.lower()
agoda_df['new_name'] = agoda_df['hotel name'].str.lower()
booking_df['new_name'] = booking_df['Hotel name'].str.lower()

In [120]:
#Bước 2: Loại bỏ những substring thông dụng: khách sạn, căn hộ,...
subnames = ["căn hộ", "apartment", 
            "khách sạn", "hotel", "hostel", "nhà khách", "nhà thuyền"
            "khu", "nghỉ dưỡng", "nghỉ mát", "resort", "spa",
            "nhà", "nhà dân dịch vụ", "home", "house", "homestay", "serviced homestay",
            "biệt thự", "villa", 
            "du thuyền", "cruise", 
            "gần", "sang trọng", "near", "luxury", "mới", 
            "bungalow", "restaurant", "residences", "studio", "bar", 
            "và", "and", "&"
            ]
agoda_df['new_name'].replace(subnames,'', inplace=True, regex=True)
booking_df['new_name'].replace(subnames,'', inplace=True, regex=True)
traveloka_df['new_name'].replace(subnames,'', inplace=True, regex=True)
mytour_df['new_name'].replace(subnames,'', inplace=True, regex=True)

In [124]:
#Bước 3: Bỏ dấu
agoda_df['new_name']=[unidecode.unidecode(y) for y in agoda_df['new_name']]
booking_df['new_name']=[unidecode.unidecode(y) for y in booking_df['new_name']]
traveloka_df['new_name']=[unidecode.unidecode(y) for y in traveloka_df['new_name']]
mytour_df['new_name']=[unidecode.unidecode(y) for y in mytour_df['new_name']]

In [125]:
#Bước 4: Với những tên KS có 2 phần tiếng anh, tiếng việt --> lấy 1 phần
agoda_df['new_name'] = [y.partition('(')[0] for y in agoda_df['new_name']]
booking_df['new_name'] = [y.partition('(')[0] for y in booking_df['new_name']]
traveloka_df['new_name'] = [y.partition('(')[0] for y in traveloka_df['new_name']]
mytour_df['new_name'] = [y.partition('(')[0] for y in mytour_df['new_name']]

In [126]:
#Bước 5: Loại bỏ khoảng cách giữa các từ
agoda_df['new_name'].replace(' ','', inplace=True, regex=True)
booking_df['new_name'].replace(' ','', inplace=True, regex=True)
traveloka_df['new_name'].replace(' ','', inplace=True, regex=True)
mytour_df['new_name'].replace(' ','', inplace=True, regex=True)

In [127]:
#Check kết quả
for i in range(100):
    print(traveloka_df['Hotel name'][i]+' ==> '+traveloka_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(booking_df['Hotel name'][i]+' ==> '+booking_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(agoda_df['hotel name'][i]+' ==> '+agoda_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(mytour_df['Hotel name'][i]+' ==> '+mytour_df['new_name'][i])

Khách sạn MK Premier Boutique ==> mkpremierboutique
Classy Holiday Hotel & Spa ==> classyholiday
La Beaute Boutique Hotel & Spa ==> labeauteboutique
Căn Hộ Wecozy ==> wecozy
Green Hotel & Apartment HN ==> greenhn
Chariot Hotel ==> chariot
Khách sạn Grand Plaza Hà Nội ==> grplazahanoi
VNAHOMES APARTHOTEL ==> vnasapart
Lucien Hanoi Hotel & Spa ==> lucienhanoi
SOJO Ga Hanoi ==> sojogahanoi
Hanoi Le Jardin Hotel & Spa ==> hanoilejardin
Mina Hotel & Spa ==> mina
Khách Sạn Halais ==> halais
Hanoi E Central Hotel ==> hanoiecentral
Hanoi Emerald Waters Hotel & Spa ==> hanoiemeraldwaters
Khách sạn Bảo Sơn ==> baoson
Romantique Hotel De Hanoi ==> romantiquedehanoi
Millennium Hanoi Hotel ==> millenniumhanoi
ISTAY Hotel Apartment 6 ==> istay6
Honeymoon Hotel and Apartment ==> honeymoon
New Style House Hotel ==> newstyle
V-Studio Hotel Apartment 3 ==> v-3
Hanoi Brilliant Hotel and Spa ==> hanoibrilliant
My Hotel 23 ==> my23
Little Hanoi Hotel ==> littlehanoi
Spring Hotel Hanoi ==> springhanoi
Khách

**Address**

In [128]:
#Bước 1: Chuyển về dạng chữ thường
traveloka_df['new_add'] = traveloka_df['Address'].str.lower()
mytour_df['new_add'] = mytour_df['Address'].str.lower()
agoda_df['new_add'] = agoda_df['address'].str.lower()
booking_df['new_add'] = booking_df['Address'].str.lower()

In [129]:
#Bước 2: Loại bỏ mã vùng/mã tỉnh 
traveloka_df['new_add'] = [y.partition(', việt nam')[0] for y in traveloka_df['new_add']]
booking_df['new_add'] = [y.partition(', việt nam')[0] for y in booking_df['new_add']]
agoda_df['new_add'] = [y.partition(', việt nam')[0] for y in agoda_df['new_add']]
mytour_df['new_add'] = [y.partition(', việt nam')[0] for y in mytour_df['new_add']]

In [130]:
#Bước 3: Loại bỏ các substring thông dụng
subadds = ["xóm", "quận", "thị xã", "ngõ", "số", "tòa", "thành phố", "phố cổ", "phố",  
           "đường", "phường", "cư xá", "cảng", "hẻm", "khu", "đô thị", "nhà",
           "ngách", "phòng", "căn", "tầng", "tỉnh", "tổ", "ap", "ấp", "lô", "bãi",
           "xã", "huyện", "du lịch", "di tích", "đại lộ", "đảo", "và", "tel 02633.608084",
           "trung tâm", "đối diện", "phía", "sau", "trước", "siêu thị", "vn", "quan",
           "no", "no.", "nos", "island", "alley", "lane", "street", "ward", "district",
           "group", "province", "block", "beach", "city", "basilica", "market", "lake",
           "area", "unit", "gate", "port", "wharft", "near by", "building", "zone",
           "tp", "str", "st", "dist", "pro", "bis", "kđt", "hcmc", "hcm", 
           "hà nội", "hồ chí minh", "đà lạt", "đà nẵng", "nha trang", "vũng tàu", 
           "phú quốc", "quảng ninh", "hạ long", "ha long", "ha noi", "ho chi minh",
           "da lat", "da nang", "vung tau", "phu quoc", "quang ninh", "viet nam", "vietnam", 
           ","
            ]

In [131]:
agoda_df['new_add'].replace(subadds,'', inplace=True, regex=True)
booking_df['new_add'].replace(subadds,'', inplace=True, regex=True)
traveloka_df['new_add'].replace(subadds,'', inplace=True, regex=True)
mytour_df['new_add'].replace(subadds,'', inplace=True, regex=True)

In [132]:
# #Bước 4: Bỏ dấu
agoda_df['new_add']=[unidecode.unidecode(y) for y in agoda_df['new_add']]
booking_df['new_add']=[unidecode.unidecode(y) for y in booking_df['new_add']]
traveloka_df['new_add']=[unidecode.unidecode(y) for y in traveloka_df['new_add']]
mytour_df['new_add']=[unidecode.unidecode(y) for y in mytour_df['new_add']]

In [133]:
#Bước 5: Loại bỏ khoảng cách giữa các từ
# agoda_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# booking_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# traveloka_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# mytour_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
agoda_df['new_add'].replace(' ','', inplace=True, regex=True)
booking_df['new_add'].replace(' ','', inplace=True, regex=True)
traveloka_df['new_add'].replace(' ','', inplace=True, regex=True)
mytour_df['new_add'].replace(' ','', inplace=True, regex=True)

In [134]:
#Check kết quả
for i in range(100):
    print(traveloka_df['Address'][i]+' ==> '+traveloka_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(booking_df['Address'][i]+' ==> '+booking_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(agoda_df['address'][i]+' ==> '+agoda_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(mytour_df['Address'][i]+' ==> '+mytour_df['new_add'][i])

72-74 Hàng Buồm, Hàng Buồm, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000 ==> 72-74hangbuomhangbuomquanhoankiem
49 Lãn Ông, Hàng Bồ, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 108313 ==> 49lanonghangboquanhoankiem
11 Ngõ Trung Yên, phố Đinh Liệt, Hoan Kiếm, Hàng Bạc, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 106876 ==> 11trungyendinhliethoankiemhangbacquanhoankiem
Số 20, Tống Duy Tân, Phố Hàng Bống, Hàng Bông, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000 ==> 20tongduytanhangbonghangbongquanhoankiem
No 5, alley 148 Tran Duy Hung street, Yen Hoa ward, Cau Giay district, Hanoi, Phường Trung Hòa, Quận Cầu Giấy, Hà Nội, Việt Nam, 100000 ==> 5148tranduyhungyenhoacaugiayhaitrunghoaquancaugiay
235 Phố Huế, Phố Huế, Quận Hai Bà Trưng, Hà Nội, Việt Nam, 100000 ==> 235phohuehuequanhaibatrung
117 Trần Duy Hưng, Phường Trung Hòa, Quận Cầu Giấy, Hà Nội, Việt Nam, 100000 ==> 117tranduyhungtrunghoaquancaugiay
30 Phố Võng Thị , Bưởi, Tây Hồ, Hà Nội, Việt Nam, 100000 ==> 30vongthibuoitayho
Số 2 , Cầu Gỗ , Hoàn Kiếm , 

# **Merge_df**

In [136]:
#Rename
#Traveloka & Mytour
traveloka_df.rename(columns = {'Min Price':'Price'}, inplace = True)
mytour_df.rename(columns = {'Min Price':'Price'}, inplace = True)

In [137]:
agoda_df.columns

Index(['city', 'hotel name', 'url', 'address', 'stars', 'price', 'rating',
       'n_reviews', 'image', 'reviews', 'facilities', 'nearby places',
       'new_name', 'new_add'],
      dtype='object')

In [139]:
#Agoda
agoda_df.columns = ["City", "Hotel name", "Url", "Address", "Stars",
                    "Price", "Rating", "Number of reviews","Image", "Reviews",
                    "Facilities", "Nearby places", "new_name", "new_add"]
agoda_df["Description"] = ""

In [140]:
#Reorder
column_order = ["City", "Hotel name", "Url", "Address", "Stars", 
                "Price", "Rating", "Number of reviews", "Reviews",	
                "Facilities",	"Description",	"Nearby places", "Image", "new_name", "new_add"]

traveloka_df = traveloka_df.reindex(columns=column_order)
mytour_df = mytour_df.reindex(columns=column_order)
agoda_df = agoda_df.reindex(columns=column_order)

In [141]:
booking_df.columns

Index(['City', 'Hotel name', 'Url', 'Address', 'Stars', 'Price', 'Rating',
       'Number of reviews', 'Reviews', 'Facilities', 'Description',
       'Nearby places', 'Image', 'new_name', 'new_add'],
      dtype='object')

In [142]:
agoda_df.columns

Index(['City', 'Hotel name', 'Url', 'Address', 'Stars', 'Price', 'Rating',
       'Number of reviews', 'Reviews', 'Facilities', 'Description',
       'Nearby places', 'Image', 'new_name', 'new_add'],
      dtype='object')

In [143]:
traveloka_df.columns

Index(['City', 'Hotel name', 'Url', 'Address', 'Stars', 'Price', 'Rating',
       'Number of reviews', 'Reviews', 'Facilities', 'Description',
       'Nearby places', 'Image', 'new_name', 'new_add'],
      dtype='object')

In [144]:
mytour_df.columns

Index(['City', 'Hotel name', 'Url', 'Address', 'Stars', 'Price', 'Rating',
       'Number of reviews', 'Reviews', 'Facilities', 'Description',
       'Nearby places', 'Image', 'new_name', 'new_add'],
      dtype='object')

In [146]:
#Concat
frames = [booking_df, agoda_df, traveloka_df, mytour_df]

In [147]:
data_res = pd.concat(frames)

In [150]:
data_res['City'].unique()

array(['Hà Nội', 'TP. Hồ Chí Minh', 'Đà Lạt', 'Đà Nẵng', 'Nha Trang',
       'Phú Quốc', 'Vũng Tàu', 'Hạ Long', 'Đảo Phú Quốc',
       'Vũng Tàu', 'Ha Long', 'Hồ Chí Minh', 'Hà Nội', 'Hạ Long'],
      dtype=object)

In [151]:
data_res['City'] = np.where(data_res['City'] == 'Hà Nội', 'Hà Nội', data_res['City'])
data_res['City'] = np.where(data_res['City'] == 'Vũng Tàu', 'Vũng Tàu', data_res['City'])
data_res['City'] = np.where(data_res['City'] == 'Đảo Phú Quốc', 'Phú Quốc', data_res['City'])
data_res['City'] = np.where(data_res['City'] == 'Hồ Chí Minh', 'TP. Hồ Chí Minh', data_res['City'])
data_res['City'] = np.where(data_res['City'] == 'Hạ Long', 'Hạ Long', data_res['City'])
data_res['City'] = np.where(data_res['City'] == 'Ha Long', 'Hạ Long', data_res['City'])

In [152]:
data_res['City'].unique()

array(['Hà Nội', 'TP. Hồ Chí Minh', 'Đà Lạt', 'Đà Nẵng', 'Nha Trang',
       'Phú Quốc', 'Vũng Tàu', 'Hạ Long'], dtype=object)

In [154]:
save_path = "/content/drive/MyDrive/20212/THDL/data/after_matching/All_data.csv"
data_res.to_csv(save_path, index=False, encoding='utf8')

In [149]:
len(data_res)

7739